# Competición Leaf de kaggle

Para realizar este ejercicio es necesario descargarse de kaggle los archivos train.csv y test.csv desde la seccion de la competicion: https://www.kaggle.com/c/leaf-classification.

Cargamos, en primer lugar, el archivo train.csv:

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('data/leaf/train.csv')
#train.head()

Tenemos 194 columnas, 1 de ellas (species) es las variable de clasificación.

Con **describe** obtenemos estadísticas básicas del dataset:

In [ ]:
train.describe()

Analizamos la varible de clasificación:

In [3]:
train.species.value_counts().describe()

count    99
mean     10
std       0
min      10
25%      10
50%      10
75%      10
max      10
Name: species, dtype: float64

*count* nos indica que tenemos 99 clases diferentes
*mean* que tenemos 10 muestras de cada clase (no muchas)

Nos vamos definir las variables globales TARGET (variable de clasificación) y TRAIN_COLUMNS (aquellas variables que vamos a utilizar para entrenar el modelo). De TRAIN_COLUMNS, al menos, deberemos excluir el *id* y la variable de clasificación:

In [4]:
TARGET = 'species'
TRAIN_COLUMNS = [c for c in train.columns 
                     if not c in ['id', 'species']]

X = train[TRAIN_COLUMNS]
y = train[[TARGET]]

Nos vamos a definir tambíen una variable para guardar el algoritmo que vamos a usar:

In [5]:
from sklearn.linear_model import LogisticRegression
ESTIMATOR = LogisticRegression(random_state=2016)

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder().fit(train.species) 
y = le.transform(train.species)           # encode species strings
classes = list(le.classes_) 
classes

['Acer_Capillipes',
 'Acer_Circinatum',
 'Acer_Mono',
 'Acer_Opalus',
 'Acer_Palmatum',
 'Acer_Pictum',
 'Acer_Platanoids',
 'Acer_Rubrum',
 'Acer_Rufinerve',
 'Acer_Saccharinum',
 'Alnus_Cordata',
 'Alnus_Maximowiczii',
 'Alnus_Rubra',
 'Alnus_Sieboldiana',
 'Alnus_Viridis',
 'Arundinaria_Simonii',
 'Betula_Austrosinensis',
 'Betula_Pendula',
 'Callicarpa_Bodinieri',
 'Castanea_Sativa',
 'Celtis_Koraiensis',
 'Cercis_Siliquastrum',
 'Cornus_Chinensis',
 'Cornus_Controversa',
 'Cornus_Macrophylla',
 'Cotinus_Coggygria',
 'Crataegus_Monogyna',
 'Cytisus_Battandieri',
 'Eucalyptus_Glaucescens',
 'Eucalyptus_Neglecta',
 'Eucalyptus_Urnigera',
 'Fagus_Sylvatica',
 'Ginkgo_Biloba',
 'Ilex_Aquifolium',
 'Ilex_Cornuta',
 'Liquidambar_Styraciflua',
 'Liriodendron_Tulipifera',
 'Lithocarpus_Cleistocarpus',
 'Lithocarpus_Edulis',
 'Magnolia_Heptapeta',
 'Magnolia_Salicifolia',
 'Morus_Nigra',
 'Olea_Europaea',
 'Phildelphus',
 'Populus_Adenopoda',
 'Populus_Grandidentata',
 'Populus_Nigra',
 'Pr

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2016)


In [9]:
ESTIMATOR.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2016, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
ESTIMATOR.predict_proba(X_test)

array([[ 0.01063669,  0.01344462,  0.00965552, ...,  0.00728016,
         0.00764327,  0.01294914],
       [ 0.0084319 ,  0.00931848,  0.00871455, ...,  0.00754165,
         0.00817185,  0.00941171],
       [ 0.01079442,  0.01144427,  0.00911849, ...,  0.0076757 ,
         0.00792944,  0.0100895 ],
       ..., 
       [ 0.009243  ,  0.01030211,  0.00840652, ...,  0.00731508,
         0.00985176,  0.00958918],
       [ 0.01110626,  0.01230008,  0.00811379, ...,  0.00732597,
         0.00841908,  0.0106242 ],
       [ 0.01142712,  0.01296547,  0.00817202, ...,  0.00721551,
         0.00854942,  0.01141994]])

*predict_proba* nos devuelve la probabilidad de que la muestra pertenezca a cada una de las clases. Dicha matriz de predicciones tendrá una dimension (nº de muestras, nº de clases)

In [11]:
ESTIMATOR.predict_proba(X_test).shape

(248, 99)

In [12]:
from sklearn.metrics import log_loss
# The mean squared error
print("\tError Log-loss: %.2f" % log_loss(y_test, 
                                ESTIMATOR.predict_proba(X_test)))



ValueError: y_true and y_pred contain different number of classes 92, 99. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [ 0  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 49 50 51 52
 53 54 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 74 75 76 77 78 80
 81 82 83 84 85 86 88 89 90 91 92 93 94 95 96 97 98]

Este error es debido a que al dividir nuestros datos en entrenamiento y prueba (train y test) en el test han caido menos menos clases de las que tenemos en el train. Para solucionar este problema podemos indicar a la función *train_test_split* que asegure que hay muestras representativas de todas las clases en ambos conjuntos con el parámetro *stratify*.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=2016)

In [14]:
ESTIMATOR.fit(X_train[TRAIN_COLUMNS], y_train)

from sklearn.metrics import log_loss
# The mean squared error
print("\tError Log-loss: %.2f" % log_loss(y_test, 
                                          ESTIMATOR.predict_proba(X_test)))

	Error Log-loss: 4.24


Podemos modificar los parámetros para intentar que nuestro modelo se ajuste mejor:

In [17]:
#'tol': [0.001, 0.0001, 0.005]
ESTIMATOR = LogisticRegression(C=200000, 
                        tol=0.006,
                        solver='newton-cg', 
                        multi_class='multinomial')
ESTIMATOR.fit(X_train[TRAIN_COLUMNS], y_train)
# The mean squared error
print("\tError Log-loss: %.2f" % log_loss(y_test, 
                                          ESTIMATOR.predict_proba(X_test)))

	Error Log-loss: 0.34


También podemos probar otros algoritmos:

In [18]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, 
                            max_depth=10, 
                            min_samples_split=0.6, 
                            min_samples_leaf=1,
                            random_state=2016)
rf.fit(X_train[TRAIN_COLUMNS], y_train)

# The mean squared error
print("\tError Log-loss: %.2f" % 
      log_loss(y_test, rf.predict_proba(X_test)))

	Error Log-loss: 3.95


# Creación del fichero de predicciones

Una vez que tenemos nuestro modelo entrenado podemos crear el fichero de predicciones para subir a kaggle. Para ello, volvemos a entrenar el modelo con todos los datos:

In [19]:
ESTIMATOR.fit(X[TRAIN_COLUMNS], y)

test = pd.read_csv('data/leaf/test.csv')

ESTIMATOR.predict_proba(test[TRAIN_COLUMNS])


array([[  9.16916807e-09,   1.49113286e-09,   2.82254873e-11, ...,
          1.25319107e-08,   2.46346158e-09,   1.07865839e-09],
       [  4.18831203e-08,   9.31506332e-08,   5.19919184e-08, ...,
          6.37082116e-06,   7.38320586e-09,   8.57723986e-06],
       [  8.95528879e-09,   9.98873577e-01,   2.94284305e-07, ...,
          7.24897742e-10,   1.28457768e-11,   6.99317173e-04],
       ..., 
       [  2.42146292e-06,   3.42388599e-08,   1.89033884e-08, ...,
          1.60553177e-07,   5.39558690e-10,   1.95781704e-07],
       [  2.34628130e-08,   1.75817581e-08,   7.35651968e-07, ...,
          2.27847517e-09,   2.22342976e-09,   8.36107556e-09],
       [  3.78195026e-14,   9.47853961e-10,   1.05472736e-11, ...,
          1.55279213e-11,   2.96456414e-13,   2.39866438e-10]])

Leemos el archivo de test y calculamos las predicciones del model para estos datos:

In [20]:
test = pd.read_csv('data/leaf/test.csv')

predictions = ESTIMATOR.predict_proba(test[TRAIN_COLUMNS])

Por ultimos construimos un nuevo dataset, unión de las predicciones y la columna *id*:

In [21]:
subm = pd.DataFrame(data=predictions, columns=classes)

subm = pd.concat((test, subm), axis=1)[['id'] + classes]
#subm.head()

Y lo guardamos, indicando no nos incluya la columnas *index* del dataframe:

In [22]:
subm.to_csv("subm_lr.csv", index=False)

## Validación cruzada

Con la validación cruzada pretendemos determinar si nuestro modelo es suficientemente generalista.

Como vimos anteriormente tenemos que usar versión estratificada:

In [23]:
from sklearn.cross_validation import StratifiedKFold

kfold = StratifiedKFold(y, 4, shuffle = True)

La función *cross_val_score* creará ajustará un modelo por cada fold (en este caso 5 modelos) y calculará el error cometido. 

Tenemos que indicar los folds y la métrica que queremos usar:

In [24]:
from sklearn.cross_validation import cross_val_score

results = cross_val_score(ESTIMATOR, X, y, cv=kfold, 
                          scoring='log_loss')
results

/usr/local/lib/python3.4/dist-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.4/dist-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.4/dist-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.4/dist-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


array([-0.20989388, -0.21531911, -0.27423294, -0.17801408])

Ahora podemos calcular el error medio y la deviación:

In [25]:
print("Error medio: %0.3f" % np.mean(results) )
print("Desviación: %0.3f" % np.std(results) )

Error medio: -0.219
Desviación: 0.035


## Optimización de parametros con GridSearchCV

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X[TRAIN_COLUMNS])


params = {'C':[1, 10, 50, 100, 500, 1000, 2000, 200000], 
          'tol': [0.001, 0.0001, 0.005, 0.006]}
log_reg = LogisticRegression(solver='newton-cg', 
                             multi_class='multinomial')
clf = GridSearchCV(log_reg, params, scoring='log_loss', 
                   refit='True', n_jobs=12, cv=5)
clf.fit(scaler.transform(X[TRAIN_COLUMNS]), y)


In [ ]:
clf.best_estimator_

In [ ]:
clf.best_score_